In [43]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from transformers import XLMRobertaModel, XLMRobertaTokenizer
import torch.nn.functional as F
import psycopg
from pgvector.psycopg import register_vector

In [45]:
conn = psycopg.connect(
    dbname="postgres_db",
    user="postgres_user",
    password="postgres_password",
    host="veld_embeddings_platform_run_server",
    port="5432"
)
register_vector(conn)
cursor = conn.cursor()
cursor.execute("SELECT version();")
print(cursor.fetchone())
#cursor.close()
#conn.close()

('PostgreSQL 17.4 on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit',)


In [21]:
#MODEL_NAME = "deepset/gbert-base"
MODEL_NAME = "dbmdz/bert-base-german-cased"
#MODEL_NAME = "FacebookAI/xlm-roberta-large"
#MODEL_NAME = "FacebookAI/roberta-large"

In [22]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)
#tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_NAME)
#model = XLMRobertaModel.from_pretrained(MODEL_NAME)
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(31102, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [23]:
def get_vectors(text):
    inputs = tokenizer(text, return_tensors="pt", add_special_tokens=True)
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.squeeze(0)
    return embeddings, tokens

In [97]:
embeddings_1, tokens_1 = get_vectors("Ich sitze auf der Bank.")
embeddings_2, tokens_2 = get_vectors("Diese Bank ist aus Holz.")
embeddings_3, tokens_3 = get_vectors("Susi arbeitet in einer internationalen Bank.")
embeddings_4, tokens_4 = get_vectors("Ich lege mein Geld bei dieser Bank in Wertpapiere an.")

In [51]:
print(embeddings_1.shape)
print(embeddings_2.shape)
print(embeddings_3.shape)
print(embeddings_4.shape)

torch.Size([9, 768])
torch.Size([8, 768])
torch.Size([10, 768])
torch.Size([15, 768])


In [26]:
print(len(tokens_1))
print(len(tokens_2))
print(len(tokens_3))
print(len(tokens_4))

9
8
10
15


In [28]:
print(tokens_1)
print(tokens_1[6])
print(tokens_2)
print(tokens_2[2])
print(tokens_3)
print(tokens_3[7])
print(tokens_4)
print(tokens_4[8])

['[CLS]', 'Ich', 'sit', '##ze', 'auf', 'der', 'Bank', '.', '[SEP]']
Bank
['[CLS]', 'Diese', 'Bank', 'ist', 'aus', 'Holz', '.', '[SEP]']
Bank
['[CLS]', 'Sus', '##i', 'arbeitet', 'in', 'einer', 'internationalen', 'Bank', '.', '[SEP]']
Bank
['[CLS]', 'Ich', 'leg', '##e', 'mein', 'Geld', 'bei', 'dieser', 'Bank', 'in', 'Wertpapier', '##e', 'an', '.', '[SEP]']
Bank


In [29]:
v_1 = embeddings_1[6]
v_2 = embeddings_2[2]
v_3 = embeddings_3[7]
v_4 = embeddings_4[8]

In [30]:
print("1, 2", F.cosine_similarity(v_1, v_2, dim=0).item())
print("1, 3", F.cosine_similarity(v_1, v_3, dim=0).item())
print("1, 4", F.cosine_similarity(v_1, v_4, dim=0).item())
print("2, 3", F.cosine_similarity(v_2, v_3, dim=0).item())
print("2, 4", F.cosine_similarity(v_2, v_4, dim=0).item())
print("3, 4", F.cosine_similarity(v_3, v_4, dim=0).item())

1, 2 0.8286746740341187
1, 3 0.8310555219650269
1, 4 0.8268762230873108
2, 3 0.7706555128097534
2, 4 0.7725297212600708
3, 4 0.8637327551841736


In [32]:
embeddings_1.shape

torch.Size([9, 768])

In [53]:
print(embeddings_1.shape)
print(len(tokens_1))

torch.Size([9, 768])
9


In [154]:
def aggregate_sub_tokens(tokens_list, embeddings_list):
    tokens_list_new = []
    embeddings_list_new = []
    t_prev = None
    e_prev = []
    for i, (t, e) in enumerate(zip(tokens_list, embeddings_list)):
        if t and t.startswith("##"):
            t_prev += t[2:]
            e_prev.append(e)
        else:
            if t_prev:
                tokens_list_new.append(t_prev)
                embeddings_list_new.append(torch.mean(torch.stack(e_prev), dim=0))
            t_prev = t
            e_prev = [e]
    tokens_list_new.append(t_prev)
    embeddings_list_new.append(torch.mean(torch.stack(e_prev), dim=0))
    embeddings_list_new = torch.stack(embeddings_list_new)
    return tokens_list_new, embeddings_list_new

In [155]:
tokens_4_new, embeddings_4_new = aggregate_sub_tokens(tokens_4, embeddings_4)

In [166]:
tokens_4

['[CLS]',
 'Ich',
 'leg',
 '##e',
 'mein',
 'Geld',
 'bei',
 'dieser',
 'Bank',
 'in',
 'Wertpapier',
 '##e',
 'an',
 '.',
 '[SEP]']

In [167]:
tokens_4_new

['[CLS]',
 'Ich',
 'lege',
 'mein',
 'Geld',
 'bei',
 'dieser',
 'Bank',
 'in',
 'Wertpapiere',
 'an',
 '.',
 '[SEP]']

In [171]:
embeddings_4.shape

torch.Size([15, 768])

In [172]:
embeddings_4_new.shape

torch.Size([13, 768])

In [173]:
def write_into_db(tokens, embeddings):
    for t, e in zip(tokens, embeddings):
        cursor.execute(
            "insert into dbmdz__bert_base_german_cased__test (word, embedding) values (%s, %s)",
            (t, e.tolist())
        )
        conn.commit()

In [175]:
write_into_db(tokens_4_new, embeddings_4_new)